# Cost and scaling model

In [4]:
from wisdem.turbine_costsse.nrel_csm_tcc_2015 import nrel_csm_mass_2015, nrel_csm_2015
from openmdao.api import Problem
import numpy as np

## Mass model

The goal of this model is too model the mass given a set of parameters about the turbine.

The key to accomplishing this is the `nrel_csm_mass_2015` class.

Here, we will do a model based on this class.

First, we instantiate the `nrel_csm_mass_2015` class. Then we make an OpenMDAO `Problem` that uses this class. Then we set the inputs for our model.

In [5]:
turbine = nrel_csm_mass_2015()
prob = Problem(turbine)
prob.setup()

prob['rotor_diameter'] = 126.0
prob['turbine_class'] = 1
prob['blade_has_carbon'] = False
prob['blade_number'] = 3    
prob['machine_rating'] = 5000.0
prob['hub_height'] = 90.0
prob['bearing_number'] = 2
prob['crane'] = True

Some of the inputs require a bit more calculation before we input them into the model, such as the rotor force inputs. Here's an example where we use max tip speed, machine rating, max efficiency, and rotor speed to set the rotor torque.

In [6]:
# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90

ratedHubPower  = prob['machine_rating']*1000. / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))

Now we are set to run the model:

In [12]:
prob.run_model()
   
print("The results for the NREL 5 MW Reference Turbine in an offshore 20 m water depth location are:")
for io in turbine._outputs:
    print(io, str(turbine._outputs[io]))

The results for the NREL 5 MW Reference Turbine in an offshore 20 m water depth location are:
sharedIndeps.machine_rating [5000.]
sharedIndeps.rotor_diameter [126.]
blade.blade_mass [18590.66820649]
hub.hub_mass [44078.53687493]
pitch.pitch_system_mass [10798.90594644]
spinner.spinner_mass [973.]
lss.lss_mass [22820.27928238]
bearing.main_bearing_mass [2245.41649102]
gearbox.gearbox_mass [43468.32086769]
hss.hss_mass [994.7]
generator.generator_mass [14900.]
bedplate.bedplate_mass [41765.26095285]
yaw.yaw_mass [12329.96247921]
hvac.hvac_mass [400.]
cover.cover_mass [6836.69]
other.other_mass [8220.65761911]
transformer.transformer_mass [11485.]
tower.tower_mass [182336.48057717]
turbine.hub_system_mass [55850.44282136]
turbine.rotor_mass [111622.44744083]
turbine.nacelle_mass [167711.70418327]
turbine.turbine_mass [461670.63220128]
